<a href="https://colab.research.google.com/github/tsai-praveen/tsai-assignments/blob/main/S4/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [73]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1)  # 1x28x28, 4x28x28,
        self.bn1 = nn.BatchNorm2d(4)  
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1)  # 4x28x28, 8x28x28
        self.bn2 = nn.BatchNorm2d(8)                # 
        self.pool1 = nn.MaxPool2d(2, 2)             # 8x28x28, 8x14x14
        self.do1 = nn.Dropout(0.1)

        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) # 8x14x14, 16x14x14
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1)# 16x14x14, 32x14x14
        self.bn4 = nn.BatchNorm2d(32)
        self.do2 = nn.Dropout(0.15)

        self.conv5 = nn.Conv2d(32, 10, 3)           # 32x14x14, 64x12x12
        self.bn5 = nn.BatchNorm2d(10)
        self.do3 = nn.Dropout(0.20)

        self.gap = nn.AvgPool2d(kernel_size=10)

    def forward(self, x):
        # print(x.shape)
        x = F.relu(self.conv1(x))
        x = self.do1(self.pool1(self.bn2(F.relu(self.conv2(self.bn1(x))))))
        x = self.do2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))
        x = self.do3(self.bn5(F.relu(self.conv5(x))))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [74]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
       BatchNorm2d-2            [-1, 4, 28, 28]               8
            Conv2d-3            [-1, 8, 28, 28]             296
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
           Dropout-6            [-1, 8, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           1,168
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 32, 14, 14]           4,640
      BatchNorm2d-10           [-1, 32, 14, 14]              64
          Dropout-11           [-1, 32, 14, 14]               0
           Conv2d-12           [-1, 10, 12, 12]           2,890
      BatchNorm2d-13           [-1, 10, 12, 12]              20
          Dropout-14           [-1, 10,

In [75]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [76]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [78]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.3176201283931732 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.66it/s] 



Test set: Average loss: 0.1493, Accuracy: 9736/10000 (97%)



loss=0.1709803193807602 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.64it/s] 



Test set: Average loss: 0.0919, Accuracy: 9812/10000 (98%)



loss=0.17287923395633698 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.48it/s] 


...


Test set: Average loss: 0.0332, Accuracy: 9901/10000 (99%)



loss=0.016569709405303 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.61it/s]   



Test set: Average loss: 0.0332, Accuracy: 9899/10000 (99%)



loss=0.05502290651202202 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.67it/s] 



Test set: Average loss: 0.0318, Accuracy: 9914/10000 (99%)

